## 8. Making a Kaggle submission

We converted the model with a better performance to a CSV file and exported for submission on Kaggle. The Random Forest model appears to be the best model, considering it gives a lower rmse value and a better r_2 score, hence our selection for submission. 

In [ ]:
predicted_y = pd.DataFrame(predictions_Rfr_test, columns = ['load_shortfall_3h'])
predicted_y.head()

In [ ]:
predicted_y.describe().round()

In [ ]:
y.describe().round()

In [ ]:
output = pd.DataFrame({'time':df_test['time']})
team_submission_Lr = output.join(submitted_y)
team_submission_Lr.to_csv('team_NM2_submission.csv', index = False)